In [1]:
import openai
import random
import time
import json
import os
with open('./movie/example_train_text.json', 'r') as file: #for Game dataset, change path to /absolute_path
    
    
    
    data = json.load(file)

# Ensure you have set your OpenAI API key
openai.api_key = 'KEY'

def get_chat_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-4",  
        messages=messages,
        temperature=0
    ) 
    return response.choices[0].message['content']

In [2]:
def get_hints(idx):
    conversation_history = []
    p1 = data[idx]['prompt']
    label = data[idx]['label']
    hints = []

    conversation_history.append({"role": "user", "content": p1})
    for delay_secs in (2**x for x in range(0, 10)):
        try:
            response = get_chat_completion(conversation_history)
            conversation_history.append({"role": "assistant", "content": response})

            if label not in response: #continue
                p2 = 'Question: ' + p1 + '\n\nChatGPT: ' + response + "\n\nNow, know that none of these answers is the target. Infer about possible mistakes in the ChatGPT's predictions."
                conversation_history.append({"role": "user", "content": p2})
                response = get_chat_completion(conversation_history)
                conversation_history.append({"role": "assistant", "content": response})
                
                p3 = 'The correct answser is ' + label + '.\n\nAnalyze why ChatGPT missed the target item from a wide variety of aspects. Explain the causes for the oversights based on earlier analysis.'
                conversation_history.append({"role": "user", "content": p3})
                response = get_chat_completion(conversation_history)
                conversation_history.append({"role": "assistant", "content": response})
                
                p4 = 'Provide short hints (imperative sentences) for AI to try again according to each point of the previous step, without information leakage of the target item.'
                conversation_history.append({"role": "user", "content": p4})
                response = get_chat_completion(conversation_history)
                list_of_hints = [each for each in response.split('\n') if len(each)>0]
                
                #check effectiveness
                for each in list_of_hints:
                    if '.' in each:
                        result = each.split('.')[-2]
                    p5 = p1 + 'Hint: ' + each
                    conversation_history = [{"role": "user", "content": p5}]
                    response = get_chat_completion(conversation_history)
                    if label in response:
                        hints.append(each)
                        print('Hint: ', each, ' passes effectiveness check.')
                return hints
            else: 
                return None
            
        except openai.OpenAIError as e:
            randomness_collision_avoidance = random.randint(0, 1000) / 1000.0
            sleep_dur = delay_secs + randomness_collision_avoidance
            print(f"Error: {e}. Retrying in {round(sleep_dur, 2)} seconds.")
            time.sleep(sleep_dur)
            continue

In [3]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "assistant", "content": prompt}]
    response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0
    )
    return response.choices[0].message["content"]

In [4]:
knowledge_base = [''] #start with no-hint
kb_size = 5 # use 5 here for some fast printouts
while len(knowledge_base) < kb_size: 
    list_of_hints = get_hints(random.randint(0,len(data)-1)) #this will return a list of hints
    if list_of_hints == None:
        continue
    for each in list_of_hints:
        p6 = "Does hint '" + each + "' convey similar ideas to any of "+ str(knowledge_base) + "? Return 1 if true, else return 0."
        for delay_secs in (2**x for x in range(0, 10)):
            try:
                response = get_completion(p6)
                if '0' in response:
                    if '.' in each:
                        result = each.split('.')[-2]
                    knowledge_base.append(result)
                    print('Hint: ', result, ' passes non-redundancy check.')
                else:
                    print('Hint: ', result, ' fails non-redundancy check.')
                break
            except openai.OpenAIError as e:
                randomness_collision_avoidance = random.randint(0, 1000) / 1000.0
                sleep_dur = delay_secs + randomness_collision_avoidance
                print(f"Error: {e}. Retrying in {round(sleep_dur, 2)} seconds.")
                time.sleep(sleep_dur)
                continue



knowledge_base = random.sample(knowledge_base, kb_size) #edge case: last sample produces more hints than kb_size


Hint:  3. Understand User Preferences: Try to infer more about the user's preferences, such as a potential interest in classic films, foreign films, or films with complex narratives.  passes effectiveness check.
Hint:  5. Consider the Time Factor: Take into account the order in which the movies were watched to capture any evolution in the user's preferences.  passes effectiveness check.
Hint:   Understand User Preferences: Try to infer more about the user's preferences, such as a potential interest in classic films, foreign films, or films with complex narratives  passes non-redundancy check.
Hint:   Consider the Time Factor: Take into account the order in which the movies were watched to capture any evolution in the user's preferences  passes non-redundancy check.
Hint:  2. Don't overlook foreign films when making recommendations.  passes effectiveness check.
Hint:  3. Try to infer personal preferences from the user's watch history, such as a potential interest in classic or foreign f

In [5]:
knowledge_base

[' Also consider movies with lower ratings or less popularity',
 " Consider the Time Factor: Take into account the order in which the movies were watched to capture any evolution in the user's preferences",
 ' Pay attention to directors and actors: Look for connections between the directors and actors of the movies the user watched and those in the candidate set',
 " Understand User Preferences: Try to infer more about the user's preferences, such as a potential interest in classic films, foreign films, or films with complex narratives",
 ' Also consider movies from other genres in the candidate set']

In [ ]:
hints = [each.strip() for each in knowledge_base]
with open('./movie/hints.json', "w") as json_file:
    json.dump(hints, json_file)